In [15]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

import time

warnings.filterwarnings('ignore')

In [16]:
from razor import Block, inputs, outputs, Pipeline
from razor.api import project_space_path
import razor
from razor import Technology, LibraryMode
from razor.core.blocks import ContainerExecutor


In [17]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [4]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1913         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [5]:
print('Load Main Data')

# We will need only train dataset
# to show lags concept
train_df = pd.read_csv(project_space_path('M5Forecasting/sales_train_validation.csv'))

# To make all calculations faster
# we will limit dataset by 'CA' state
train_df = train_df[train_df['state_id']=='CA']


Load Main Data


In [6]:
train_df.shape

(12196, 1919)

In [7]:
train_df.iloc[:10]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


In [8]:
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
train_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

train_df[train_df['id']=='HOBBIES_1_001_CA_1_validation'].iloc[:10]


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
12196,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0
24392,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0
36588,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0
48784,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0
60980,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_6,0
73176,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_7,0
85372,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_8,0
97568,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_9,0
109764,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_10,0


In [9]:
# train_df.to_csv(project_space_path('M5Forecasting/train_df.csv'), index=False)



In [5]:
@inputs.atomic.string('path')
class HeavyOps(Block):
    def run(self, path):
        import logging
        import pandas as pd
        
        train_df = pd.read_csv(path)
        
#         # group and shift in loop
#         temp_df = train_df[['id','d',TARGET]]

#         start_time = time.time()
#         for i in range(1,8):
#             print('Shifting:', i)
#             temp_df['lag_'+str(i)] = temp_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(i))

#         print('%0.2f min: Time for loops' % ((time.time() - start_time) / 60))
        
        # Or same in "compact" manner
        LAG_DAYS = [col for col in range(1,8)]
        temp_df = train_df[['id','d',TARGET]]

        start_time = time.time()
        temp_df = temp_df.assign(**{
                '{}_lag_{}'.format(col, l): temp_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
                for l in LAG_DAYS
                for col in [TARGET]
            })
        
        logging.info('%0.2f min: Time for bulk shift')
        logging.info(time.time() - start_time / 60)
        print('%0.2f min: Time for bulk shift' % ((time.time() - start_time) / 60))
        
        temp_df.to_csv(project_space_path('M5Forecasting/temp_df.csv'), index=False)
        

In [6]:
heavy_ops = (
    HeavyOps()
    .path('M5Forecasting/train_df.csv', adapter=project_space_path)  # (series, string)
    .executor(ContainerExecutor(cores=2, memory=8192))
)


In [7]:
p = Pipeline("Heavy Operation", targets=[heavy_ops], env=Technology.PYTHON)  #preprocess


In [8]:
deployed_pipeline = razor.api.engines('DS-engine').execute(pipeline=p)


In [9]:
deployed_pipeline

PlatformPipelineRun(project_id='c0b26d85-a4c9-44f5-9d0e-f540dd2de644', pipeline_id='35c8cf60-d894-11ea-bdec-8be76246937b', pipeline_name='Heavy Operation', pipeline_run_id='0a746093-199a-4495-be10-08b1ee274b35', pipeline_version=None, comment=None, created_on='2020-08-07T09:56:06.486+0000', start_time=None, end_time=None, eta=0, run_at=1596794166486, run_duration=0, compute_time=0, wait_time=0, ran_by_user=RanByUser(user_name='Ayan Basak', email='ayan.basak@razorthink.com'), status='IN_PROGRESS', block_status=[PlatformPipelineBlockRun(pipeline_run_id='0a746093-199a-4495-be10-08b1ee274b35', pipeline_name='Heavy Operation', pipeline_status='IN_PROGRESS', block_id='ff32d553-e9db-4f57-9080-bd61bc70ad4e', block_run_id='d79f6b4c-8aa0-423d-8c09-7c1e0828ff06', block_name='HeavyOps_1', resource_spec=ResourceAllocated(cores=2, memory=8192, use_gpu=False, gpu=0, use_gpu_if_available=False, additional_resource_spec={}, run_env='PYTHON'), technology='PYTHON', status='READY_TO_GO', containers=[], cluster_id='ff32d553-e9db-4f57-9080-bd61bc70ad4e', _log=None, _metric=None, wait_time=0, compute_time=0, start_time=None, end_time=None, queued_at='2020-08-07T09:56:07.000+0000')], run_number=58, pipeline_variable_list=[], block_run_details=[BlockRunDetail(block_id='ff32d553-e9db-4f57-9080-bd61bc70ad4e', block_run_id='d79f6b4c-8aa0-423d-8c09-7c1e0828ff06', block_name='HeavyOps_1', technology='PYTHON', log_path=None, input_parameters='{\n  "path": {\n    "dtype": "string",\n    "required": "false",\n    "series": false,\n    "adapter": "eJxrYJkazgABPUpFiVX5RXqJBZl6yflFqXopiSWJ8cX5pUXJqfG5iXmJ6alFU3p0AnISS9Lyi3JdgLLBYElfiJxeQVF+VmpySXxxQSJQR0FiScaUyVP0AGu0JDo=",\n    "validator": "eJxrYJnqyAABPRJFiVX5RXrJ+UWpekk5+cnZxXqZeQWlJcVTehQcS/JzM5OdQKLBJUWZeemeIBm9ssSczJTEkvyiKZOn6AEArzAbbQ==",\n    "source_id": null,\n    "source_name": null,\n    "inferred_type": "STATIC",\n    "value": "eJxrYJkqxwABPVK+pm75RanJicUlmXnp+iVFiZl58SlpesnFZVP0ANu4DIc="\n  }\n}', output_parameters=None, status='READY_TO_GO', created_at='2020-08-07T09:56:06.528+0000', queued_at='2020-08-07T09:56:07.000+0000', start_time=None, end_time=None, compute_time=0, wait_time=0, run_time=0, block_run_output_list=[], resource_allocated=ResourceAllocated(cores=2, memory=8192, use_gpu=False, gpu=0, use_gpu_if_available=False, additional_resource_spec={}, run_env='PYTHON'))], pipeline_json='{\n  "id": "35c8cf60-d894-11ea-bdec-8be76246937b",\n  "projectId": "c0b26d85-a4c9-44f5-9d0e-f540dd2de644",\n  "name": "Heavy Operation",\n  "description": "",\n  "blocks": [\n    {\n      "name": "HeavyOps_1",\n      "id": "ff32d553-e9db-4f57-9080-bd61bc70ad4e",\n      "artifact_id": null,\n      "class_name": "HeavyOps",\n      "code": "eJx9VVtPG0cUnlnfFhNo1BACqoTcKJGgCq6pMIIKoVLTpu02ThV4aBWh0Xhnb/F4d9mdpSXYUiUgAWlaqcr03bz31/UPtGfWJvBA6wfv7LnMOd83Z779tfjn32WU/+QDm0cZiwO7y536jbWS8yRx/COWUOGQtOtwR0QhsTlNU/VOLfpIzpAe7d7mk2YnC7gIwlTJojiKHTBK8xuHHh49j8E2l9DXUVK3o8Spd3hkd9PxQ8nSl3oB8WdqoGSFEBbZhKh2Kmvra4211XV3g3VW7dWVdWejsdbYoDZdBdNKs9FUbaFeQNairBLSi1jGHciUJrzQINTLQpKFChqfJm7AOXGz0BZBFI7hfHANJ3eBGaxTZIyFvAfSipizp1/OoN6iZVjIqlglq9D6i2GGOBoY+RMPCv3C0OjjSzwoMoMVTpEosCIrnRg76AQPSv0iK7OKKHqFeXSBRGlYukSD8jl6VXbRW+RiZrKJU7Nf0tG/4wvUN4YVVr3EOF+N4vtlNvkRmkPaKkxWvbLOIW2fhUyMYMcJUWV3TjCbYtO/GRgxtIvUYttC/sKihaF/wypYu627HuqjfXOA+/gP46AIMUtL0qg3gDk74uqNkgubQXwk/ChcDsI4E8vNZXttrbPSdCnbcFe3lKxu8iAVdtSLt5Q10zLKeg99LrJ2df51OIX6Jhwz5elW/WY8tkxpBExJzNR1Z6Z1r/VFoDtbB5aQh3fQ/pMBPsdDYNkDdvt4Hg2NnDHMjFPzErPCsNDH5+jSeFc4mM6RLPqfXG2JAey9VrWPhkDNJQZn+wwmL/UDV8CZSvwL/PuPpLnJaa/DKHQ230Jg52AfgVn6DzAssEdg6u9z5cTxgHDqkeOBEjCc5b3tF0+/2oPh9pIoiztHECESGqZulPSULOsHFRDqf+w/fKP7qF5vq6y5VrFsGBjShdOLCXN1r80z9b8cX+fLhceN+mdurReEn9f2gp5Tg3q1Tsa7tRF+a1POP2t+DVfTpimMvffpuE7dTg/VhSwFIXM0QRpLhUeeBzHQdkxDRuECm4lDGdGxsgSoPEf5myACUAmCQB8CLwRRCEI3goA4CUIBdhGNMj6Mk+iVYwuSxtR2SEyFn9cppg53IS03+M+lEcOQmMAaXEtgQJrfbz8lO9s/7cIApoImguQFgcNH/n24mHMIm9g0qlMzeNow8V2jWqziWTxnqCV/Z8Tdd//AT4vHJIG6dpd6Wj3aWj5C2tNrfyZ7OxaYisejDlCrX/wf5cORntE4GGkao4KSNMoSwNCjIWyVKPnkB06FPtwd8O

In [ ]:
temp_df = pd.read_csv('M5Forecasting/temp_df.csv')




Shifting: 1


# Original Lag Code

In [18]:
@inputs.atomic.string('grid_path')
class LagCreator(Block):
    def run(self, grid_path):
        from razor.api import project_space_path

        ########################### Apply on grid_df

        grid_df = pd.read_pickle(grid_path)

        TARGET = 'sales'         # Our main target
        END_TRAIN = 1913         # Last day in train set
        MAIN_INDEX = ['id','d']  # We can identify item by these columns
        
        # We need only 'id','d','sales'
        # to make lags and rollings
        grid_df = grid_df[['id','d','sales']]
        SHIFT_DAY = 28

        # Lags
        # with 28 day shift
        start_time = time.time()
        self.logger.info('Create lags')

        LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
        grid_df = grid_df.assign(**{
                '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
                for l in LAG_DAYS
                for col in [TARGET]
            })

        # Minify lag columns
        for col in list(grid_df):
            if 'lag' in col:
                grid_df[col] = grid_df[col].astype(np.float16)

        self.logger.info('%0.2f min: Lags' % ((time.time() - start_time) / 60))

        # Rollings
        # with 28 day shift
        start_time = time.time()
        self.logger.info('Create rolling aggs')

        for i in [7,14,30,60,180]:
            self.logger.info('Rolling period:', i)
            grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
            grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

        # Rollings
        # with sliding shift
        for d_shift in [1,7,14]: 
            self.logger.info('Shifting period:', d_shift)
            for d_window in [7,14,30,60]:
                col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
                grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)


        self.logger.info('%0.2f min: Lags' % ((time.time() - start_time) / 60))

        self.logger.info('Save lags and rollings')
        grid_df.to_pickle(project_space_path(f'M5Forecasting/lags_df_{str(SHIFT_DAY)}.pkl'))


In [19]:
lag_creator = (
    LagCreator()
    .grid_path('M5Forecasting/m5-simple-fe/grid_part_1.pkl', adapter=project_space_path)  # (series, string)
    .executor(ContainerExecutor(cores=2, memory=20000))
)

In [20]:
p = Pipeline("lag_creator", targets=[lag_creator], env=Technology.PYTHON)  #preprocess


In [21]:
deployed_pipeline = razor.api.engines('DS-engine').execute(pipeline=p)


In [22]:
deployed_pipeline

PlatformPipelineRun(project_id='c0b26d85-a4c9-44f5-9d0e-f540dd2de644', pipeline_id='efab35a8-d894-11ea-bdec-8be76246937b', pipeline_name='lag_creator', pipeline_run_id='8b2e0132-91b7-4e0a-b1cc-b2074e09e01a', pipeline_version=None, comment=None, created_on='2020-08-07T10:01:18.462+0000', start_time=None, end_time=None, eta=0, run_at=1596794478462, run_duration=0, compute_time=0, wait_time=0, ran_by_user=RanByUser(user_name='Ayan Basak', email='ayan.basak@razorthink.com'), status='IN_PROGRESS', block_status=[PlatformPipelineBlockRun(pipeline_run_id='8b2e0132-91b7-4e0a-b1cc-b2074e09e01a', pipeline_name='lag_creator', pipeline_status='IN_PROGRESS', block_id='5e94f374-a9a1-4785-82d1-b6b2b1fd0a80', block_run_id='05e41f8b-b774-4914-92f2-a89188520509', block_name='LagCreator_1', resource_spec=ResourceAllocated(cores=2, memory=20000, use_gpu=False, gpu=0, use_gpu_if_available=False, additional_resource_spec={}, run_env='PYTHON'), technology='PYTHON', status='READY_TO_GO', containers=[], cluster_id='5e94f374-a9a1-4785-82d1-b6b2b1fd0a80', _log=None, _metric=None, wait_time=0, compute_time=0, start_time=None, end_time=None, queued_at='2020-08-07T10:01:19.000+0000')], run_number=60, pipeline_variable_list=[], block_run_details=[BlockRunDetail(block_id='5e94f374-a9a1-4785-82d1-b6b2b1fd0a80', block_run_id='05e41f8b-b774-4914-92f2-a89188520509', block_name='LagCreator_1', technology='PYTHON', log_path=None, input_parameters='{\n  "grid_path": {\n    "dtype": "string",\n    "required": "false",\n    "series": false,\n    "adapter": "eJxrYJkazgABPUpFiVX5RXqJBZl6yflFqXopiSWJ8cX5pUXJqfG5iXmJ6alFU3p0AnISS9Lyi3JdgLLBYElfiJxeQVF+VmpySXxxQSJQR0FiScaUyVP0AGu0JDo=",\n    "validator": "eJxrYJnqyAABPRJFiVX5RXrJ+UWpekk5+cnZxXqZeQWlJcVTehQcS/JzM5OdQKLBJUWZeemeIBm9ssSczJTEkvyiKZOn6AEArzAbbQ==",\n    "source_id": null,\n    "source_name": null,\n    "inferred_type": "STATIC",\n    "value": "eJxrYJmqxwABPVq+pm75RanJicUlmXnp+rmmusWZuQU5qbppqfrpRZkp8QWJRSXxhnoF2TlT9ADQohIn"\n  }\n}', output_parameters=None, status='READY_TO_GO', created_at='2020-08-07T10:01:18.528+0000', queued_at='2020-08-07T10:01:19.000+0000', start_time=None, end_time=None, compute_time=0, wait_time=0, run_time=0, block_run_output_list=[], resource_allocated=ResourceAllocated(cores=2, memory=20000, use_gpu=False, gpu=0, use_gpu_if_available=False, additional_resource_spec={}, run_env='PYTHON'))], pipeline_json='{\n  "id": "efab35a8-d894-11ea-bdec-8be76246937b",\n  "projectId": "c0b26d85-a4c9-44f5-9d0e-f540dd2de644",\n  "name": "lag_creator",\n  "description": "",\n  "blocks": [\n    {\n      "name": "LagCreator_1",\n      "id": "5e94f374-a9a1-4785-82d1-b6b2b1fd0a80",\n      "artifact_id": null,\n      "class_name": "LagCreator",\n      "code": "eJyVVm1z28YRvgNAEqJkyvGLXhzXYZvJlG5jVrIti8lwPFEox3Fg84Okxk1mPDcQDuDBAgEIOMRSQnYytR1SyU2mnVw/dTql/kD7pT+p/6PdA0lLTuK05Qfe3R5u99nndm/3S+PPgzLKf2LRCaKMxr6zF7j1U3MplkniskOa2Nwl6Z4buDwKiRPYaSq/kzWGxAXStfd+bE+Yu5kfcD9MpTD4YeyCUJTv251W4to8SqRYSuzPo6TuRIlb3w0iZy+dDFIU3lcTOPFc9qUoEUIjhxDZTkX1pr1KV+kN17vpNW6uu+v2+upKg95quM769evemiPbXG7BqZooE9KNaBa4cFKYsLD9UE31JAslQK8Qzw8C4mWhw/0onDg0f+JQvgVikJ4hE2/I1BWzFVF3Ry2eg72apVnImrFKlv7+JQ0hiqkWoC7uaxhxbaT38DE+Mqh+hKnR12mBFjta3xgaakb1jr6MYLd0hLgxMo5Rv9BDjwujIjWPMUZ0hpafIV4aoiGis4voqbaJnuJ+cWg8Nj00wAPD0+gcPfPM7BXVzrdg6eBtPjM0nuJN9KjaL9FKr7RXTL4eGr3SMhqV+ezjuWOsVk20//VDNLE2P7beKywhevZ1tICU9e8hek3JDv5Az4Hmz4+m0vPDwjF4OjRGZ2ihA74OMdipDNCgANgu0IvPwJMTu3SBzw8LT/Eiar7qzOIPziydnNn/10N0ENNlwMCOtAmGS0NNYTjYoq+D/KMjnV6GE5o6QX+2iGCuq3nf/IFFbaCDxSvft9gzm+iPeP/Lh+qf/TRLk7031Bz0n+1ptAoW4Tb+gujP/wqW85tE20jW2hYS5+Ikeuw6nKSx7bgktjmDOBKF1A7cVD44LAnNp1JgKq3LYjZPGbca2J1Usis1C0OsaZZubbfOdsD0I7OPe/hP2r4B6q9eFVp9BaLciQL5lRRvNP34kLMovOaHccavrTau7b7juWsr3vqNtZX125CUzcBPuRN149vSWmppRaVEJZH4xUm61iFp6k3ISjtIb9dPn5g/QWRa51vv+QpR4wgNcUeF39t9PNRG4LoivKcIxwPkYZUgim6qj7QeHqJj7Tt9v5J7UGNvTlVicPJ8q9xDI4j9Ywyb7Zwl5ntc8YUP4J/9SpjNwO7uUhvgXGohkAcgH/vw61f6QH1n7EP9xWkx80WfAM3ki77k8PB0kiiLdw9BzhM7TL0o6UpRVIPN5UCyX7LaV8p++USZtJZbRlHTMHy4s7F1985OrsenhHpwH+yd5/K/cHu